In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
print(np.__version__)

1.26.4


In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [8]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [9]:
dependent=dataset['Purchased']

In [10]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
grid.fit(X_train_, y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1', verbose=3)

In [15]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test_) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions) 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [28]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100}: 0.9256751954513149


In [29]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [ 4 45]]


In [30]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [31]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.5096038415366146

In [32]:
table=pd.DataFrame.from_dict(re)

In [33]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001585,0.001473,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.002913,0.003613,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.041011,0.006698,0.010215,0.003186,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.764706,0.769231,0.780488,0.871795,0.918919,0.821028,0.062703,7
3,0.282814,0.013220,0.012005,0.005721,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.800000,0.810811,0.809524,0.926829,0.923077,0.854048,0.058026,2
4,0.044926,0.010071,0.006892,0.003693,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.687500,0.871795,0.780488,0.871795,0.923077,0.826931,0.083508,6
5,0.290508,0.018823,0.014473,0.002542,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.727273,0.810811,0.809524,0.900000,0.923077,0.834137,0.070453,5
6,0.001284,0.002568,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000931,0.000531,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.031427,0.006201,0.006863,0.002143,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.800000,0.777778,0.809524,0.900000,0.894737,0.836408,0.050857,4
9,0.284326,0.015191,0.010071,0.006890,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.727273,0.810811,0.809524,0.900000,0.947368,0.838995,0.076956,3


In [34]:
age_input=float(input("Age:"))
EstimatedSalary_input=float(input("EstimatedSalary:"))
Gender_Male_input=float(input("Gender_Male:"))

Age: 14
EstimatedSalary: 123
Gender_Male: 12


In [35]:
Future_Prediction = grid.predict([[age_input,EstimatedSalary_input,Gender_Male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
